In [ ]:
!pip install PuLP

In [ ]:
import numpy as np
import pulp
import re

# Function to calculate Euclidean distance between two cities
def euclidean_distance(c1, c2):
    return np.sqrt((c1[0] - c2[0])**2 + (c1[1] - c2[1])**2)

# Function to generate distance matrix
def calculate_distances(coords):
    num_cities = len(coords)
    dist_matrix = {}
    for i in range(num_cities):
        for j in range(num_cities):
            if i != j:
                dist_matrix[i, j] = np.linalg.norm(coords[i] - coords[j])
    return dist_matrix

# Define coordinates for the cities
coordinates = np.array([
    [30, 40],  # City 0
    [40, 70],  # City 1
    [70, 40],  # City 2
    [50, 30],  # City 3
    [60, 70],  # City 4
    [80, 50]   # City 5
])

# Calculate distances between all pairs of cities
distances = calculate_distances(coordinates)

# Number of cities
num_cities = len(coordinates)

# Create the problem variable to contain the problem data
prob = pulp.LpProblem("Traveling_Salesman_Problem", pulp.LpMinimize)

# Create variables x_ij where i, j are city indices
x = pulp.LpVariable.dicts("x", ((i, j) for i in range(num_cities) for j in range(num_cities) if i != j), cat='Binary')

# Objective function to minimize total distance
prob += pulp.lpSum([distances[i, j] * x[i, j] for i in range(num_cities) for j in range(num_cities) if i != j])

# Constraints ensuring each city is entered and left exactly once
for j in range(num_cities):
    prob += pulp.lpSum(x[i, j] for i in range(num_cities) if i != j) == 1, f"enter_city_{j}"
for i in range(num_cities):
    prob += pulp.lpSum(x[i, j] for j in range(num_cities) if i != j) == 1, f"leave_city_{i}"

# Solve the problem using PuLP's choice of solver
prob.solve()

# Check the status of the solution
print("Status:", pulp.LpStatus[prob.status])
print("The minimum distance to travel by visiting all cities is: {:.2f}".format(pulp.value(prob.objective)))

# Extract the route from the solution
route = []
for v in prob.variables():
    if v.varValue == 1 and 'x' in v.name:
        indices = tuple(map(int, re.findall(r'\d+', v.name)))
        route.append(indices)

# Sorting the route to find the order
if route:
    ordered_route = [route.pop(0)]
    while route:
        last = ordered_route[-1][1]
        for i, pair in enumerate(route):
            if pair[0] == last:
                ordered_route.append(route.pop(i))
                break

    # Creating a list of visited cities in order
    ordered_cities = [ordered_route[0][0]] + [i[1] for i in ordered_route]
    print("Optimal Route:", ordered_cities)
